In [1]:
#matplotlib inline
import mpld3
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#from scipy.signal import peak_prominences, find_peaks
#from scipy.stats import zscore


Im folgenden ist die KLasse "Evaluate" gezeigt, die Methoden enthält um die zu verarbeitenden Daten zu laden und aus den GITT Messungen Diffusionskoeffizienten zu berechnen
Die Datei die auszuwerten ist, wird als Klasseninstanz definiert, Das Attribut "place" wird vergeben und die Methode Find_GITT kann auf die Instanz angewendet werden. Weitere Methoden könnten folgen...

In [2]:
class Evaluate:                             
    def __init__(self, place):
        self.place = place                                                                              #Es muss ein Speicherort angegeben werden auf dem die Daten liegen 'place'
    

    
    indexdict = {}                          
    Data_subset = pd.DataFrame()

    def create(self, *args: str):
        
        Data = pd.read_csv(self.place, sep = '\t', header = 4, thousands = ',', encoding = 'utf-8')       # txt file einlesen, header muss je nach file angepasst werden
        Evaluate.Data_subset = Data[['Rec','Test Time (sec)', 'Step Time (sec)',                          # definiere welche spalten ausgelesen werden
       'Capacity','Current', 'Voltage', 'Aux #1']]                                   
        # for pos in list(args):                                                                          # *args nimmt strings auf, die als Zeilennamen gelesen und gespeichert werden
        #     key = str(pos)                                                                              # werden in einem dict gespeichert
        #     value = Data[pos].values                                                                    # Die For Schleife ist dazu gedacht über *args andere Spaltennamen auszuwählen. Habe hier halt immer dieselben genommen, deswegen de Definition darüber
        #     #print(type(value), value.ndim)
        #     Evaluate.indexdict[key] = value
        

    def Ident_GITT(self, Pulse_time, Relaxation_time):
        Rec = Evaluate.Data_subset['Rec'].to_numpy('int')                                                  # Die Spalten der .txt werden in numpy arrays umgeschrieben. 
        Current = Evaluate.Data_subset['Current'].to_numpy('float')
        Voltage = Evaluate.Data_subset['Voltage'].to_numpy('float')
        Test_Time = Evaluate.Data_subset['Test Time (sec)'].to_numpy('float')
        Step_Time = Evaluate.Data_subset['Step Time (sec)'].to_numpy('float')
        Capacity = Evaluate.Data_subset['Capacity'].to_numpy('float')
        Aux1 = Evaluate.Data_subset['Aux #1'].to_numpy('float')
        sprung = []
        sprung_zeiten = []
        sprung_zeiten_indizes = []
        Q = []
        v = []                                                                                              #Hier werden nur einige weitere Variablen definiert, die für die Berechnung der Diffusionskoeffizienzen nötig sind
        v_aux = []
        D_FC = []
        D_Aux = []
        OCV = []
        F = 96485 # [C/mol] Faraday constant
        V_m = 3.102*10**(-5) # [m^3/mol] molar Volume of the electrode
        S = 5.4*10**(-3) #[m^2] electrode contact area, estimation of area
        n_m = 3.597*10**(-4) # number of moles 
        
        for point in range(len(Rec)):                                                                  #Hier werden die "Step time" werte mit der Strompulszeit verglichen. dadurch habe ich den Anfang eines Strompulses und damit eines GITT Pulses gefunden
            if Step_Time[point] == Pulse_time:                                                         # Step Time Werden ab jedem neuen Schritt in der Prozedur aufgezeichnet
                sprung_zeiten.append(Test_Time[point]- Pulse_time)
                sprung_zeiten_indizes.append(point)                                                    #Diese wurden dann in sprung_zeiten gespeichert. Die Indizes des Array habe ich auch gespeichert aber nicht mehr
        

        GITT_Step = []                                                                                 #GITT_Step speichert die Startzeitpunkte der einzelnen GITT-Titration-Steps. Differenz zum nächsten step sollte immer sein: Stromzeit + Relaxationszeit    
        for j in range(len(sprung_zeiten)) :
            if (sprung_zeiten[j] - sprung_zeiten[j-1]) != float(Pulse_time + Relaxation_time):
                continue
            # elif j < len(sprung_zeiten) :
            #     new_sprung_zeiten.append(sprung_zeiten[j-1])
            else:
                GITT_Step.append(sprung_zeiten[j])
                
        GITT_Step = np.array(GITT_Step) #Liste -> Numpy Array
    
        for point,value in enumerate(GITT_Step):                                                        # Liste schreiben in denen die zu den Zeitschritten dazugehörigen Spannungen stehen sodass ich dann über spannungsdifferenzen abschärzen kann, was lade und was entlade       
            v.append(Voltage[Test_Time == value])                                                       #schritt ist und den Diffusionskoefizienten berechnen kann
        
        v = np.array(v).ravel()                                                                         #Das sind die Vollzell-Spannungswerte
        
        for point, value in enumerate(GITT_Step):
            v_aux.append(Aux1[Test_Time == value])

        GITT_Aux_Steps = []
        GITT_FC_Steps = []                                                                              #GITT_Aux_Steps und GITT_FC_Steps werden dann in einem array die arrays mit den Spannungswerten speichern, die zu einem einzelnen GITT Schritt hören    
        #Aux = Arbeitselektrodenspannung, ermittelt durch ref. Elektrode, FC = Fullcell Spannung                                                                                                    
                                                                                                        
        for point in range(len(v)):                 
            if point +1 < len(v):
                GITT_FC_Steps.append(Voltage[(Test_Time <= GITT_Step[point + 1]) & (Test_Time >= GITT_Step[point])])
        
        for point in range(len(v_aux)):
            if point +1 < len(v_aux):
                GITT_Aux_Steps.append(Aux1[(Test_Time <= GITT_Step[point + 1]) & (Test_Time >= GITT_Step[point])])
        
        

        for element in range(len(GITT_FC_Steps)):
            if GITT_FC_Steps[element][-1] - GITT_FC_Steps[element][0] > 0:
                delta_Es = GITT_FC_Steps[element][-1] - GITT_FC_Steps[element][0]
                delta_Et = np.max(GITT_FC_Steps[element])-GITT_FC_Steps[element][10]
                Diff_coeff = ((4 / (np.pi * 900)) * ((n_m * V_m / S) ** 2) * (delta_Es / delta_Et))
                D_FC.append(Diff_coeff)                                                                        #In D und D_Aux sind die Diffusionskoeffizienten gespeichert, bezeichnung entsprechend dem Kommentar hier drüber
                OCV.append(GITT_FC_Steps[element][-1])
                # print(Diff_coeff, GITT_Step[element])                                                     #Die ganzen Print befehle ab hier waren nur um zu schauen was heraus kommt 
                # print(Diff_coeff, GITT_Titration_Steps[element][-1])
                # print(GITT_Titration_Steps[element][-1])

                delta_Es = GITT_Aux_Steps[element][-1] - GITT_Aux_Steps[element][0]
                delta_Et = np.max(GITT_Aux_Steps[element])-GITT_Aux_Steps[element][10]
                Diff_Coeff_Aux = ((4 / (np.pi * 900)) * ((n_m * V_m / S) ** 2) * (delta_Es / delta_Et))
                D_Aux.append(Diff_Coeff_Aux)
                # print('Laden', Diff_Coeff_Aux)
                # print(Diff_Coeff_Aux)
            else:
                delta_Es = GITT_FC_Steps[element][0]-GITT_FC_Steps[element][-1]
                delta_Et = GITT_FC_Steps[element][10] - np.min(GITT_FC_Steps[element])
                Diff_coeff = ((4 / (np.pi * 900)) * ((n_m * V_m / S) ** 2) * (delta_Es / delta_Et))
                D_FC.append(Diff_coeff)
                OCV.append(GITT_FC_Steps[element][-1])
                # print(Diff_coeff, GITT_Step[element])
                # print(Diff_coeff, GITT_Titration_Steps[element][-1])
                # print(GITT_Titration_Steps[element][-1])

                delta_Es = GITT_Aux_Steps[element][0] - GITT_Aux_Steps[element][-1]
                delta_Et = GITT_Aux_Steps[element][10]- np.min(GITT_Aux_Steps[element])
                Diff_Coeff_Aux = ((4 / (np.pi * 900)) * ((n_m * V_m / S) ** 2) * (delta_Es / delta_Et))
                D_Aux.append(Diff_Coeff_Aux)
                # print('Entladen',Diff_Coeff_Aux)
                # print(Diff_Coeff_Aux)


        # plt.figure()
        # plt.plot(Test_Time[(GITT_Step[0] < Test_Time) & (Test_Time < GITT_Step[-1])], Voltage[(GITT_Step[0] < Test_Time) & (Test_Time < GITT_Step[-1])])
        # plt.show()

        # plt.figure()
        # plt.plot(Test_Time, Voltage)
        # plt.show()
        # for i in range(len(GITT_Step)):
        #     plt.figure()
        #     plt.grid = True
        #     plt.xlabel("Zeit / sqrt(s)")
        #     plt.ylabel("Spannnung / V")
        #     plt.plot(np.sqrt((Test_Time[(GITT_Step[i-1] < Test_Time) & (Test_Time < GITT_Step[i]-7200)])), Voltage[(GITT_Step[i-1]  < Test_Time) & (Test_Time < GITT_Step[i]-7200)], '.')
        #     plt.show()
        
        # plt.figure()
        # plt.plot(D)
        # plt.show()
        # plt.figure()
        # plt.plot(OCV)
        # plt.show()

        # plt.figure()
        # plt.plot(D_Aux)
        # plt.show()
        return np.asarray(D_FC)
    
    
    

        

dir = r"C:\Users\Rasmus Bewer\OneDrive\Dokumente\Sommersemester 2023\Bachelorarbeit\Daten\Messreihen GITT\BA_RB_GITT\230714_K22E21_A23D09_5hrest_form_GITT@0.1C.024.txt"
file = Evaluate(place = dir)
file.create('Test Time (sec)','Step Time (sec)', 'Current', 'Voltage', 'Aux #1')
file.Ident_GITT(Relaxation_time=7200,Pulse_time=1200)
mpld3.enable_notebook()





[array([3.394369, 3.401846, 3.407645, ..., 3.413138, 3.412833, 3.412833]), array([3.412833, 3.420768, 3.425803, ..., 3.439994, 3.439994, 3.440146]), array([3.440146, 3.447166, 3.450217, ..., 3.480125, 3.479973, 3.479973]), array([3.479973, 3.486534, 3.488212, ..., 3.517967, 3.517967, 3.517662]), array([3.517662, 3.523766, 3.524987, ..., 3.551232, 3.551232, 3.551232]), array([3.551232, 3.556725, 3.557336, ..., 3.572137, 3.572137, 3.572137]), array([3.572137, 3.577325, 3.57763 , ..., 3.58938 , 3.589227, 3.589227]), array([3.589227, 3.593347, 3.594415, ..., 3.602197, 3.602045, 3.602197]), array([3.602197, 3.606775, 3.607233, ..., 3.61471 , 3.614405, 3.614405]), array([3.614405, 3.619135, 3.619287, ..., 3.626459, 3.626459, 3.626154]), array([3.626154, 3.630884, 3.631037, ..., 3.638056, 3.638209, 3.638209]), array([3.638209, 3.642329, 3.643091, ..., 3.650874, 3.650874, 3.650874]), array([3.650874, 3.655146, 3.655451, ..., 3.664912, 3.665064, 3.664607]), array([3.664607, 3.669184, 3.669337, 